In [ ]:
import requests
from io import BytesIO
# from openai import OpenAI

# client = OpenAI()

def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
    )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
                )
    print(result.id)
    return result.id

# Replace with your own file path or URL

file_id = create_file(client, "./datas/howto-sockets.pdf")

file-2RmWSjR5STnJxWVZVnE6aW


In [2]:
vector_store = client.vector_stores.create(
    name="knowledge_base"
    )

print(vector_store.id)

vs_685cc7b331c88191881fe9376c4fe1ae


In [4]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
    )

VectorStoreFile(id='file-2RmWSjR5STnJxWVZVnE6aW', created_at=1750911024, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cc7b331c88191881fe9376c4fe1ae', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [5]:
result_list = client.vector_stores.files.list(
    vector_store_id = vector_store.id
)

print(result_list)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-2RmWSjR5STnJxWVZVnE6aW', created_at=1750910899, last_error=None, object='vector_store.file', status='completed', usage_bytes=29112, vector_store_id='vs_685cc7b331c88191881fe9376c4fe1ae', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-2RmWSjR5STnJxWVZVnE6aW', last_id='file-2RmWSjR5STnJxWVZVnE6aW')


In [12]:
response = client.responses.create(
    model = "gpt-4.1",
    input = "소켓 만드는 방법을 간단하게 설명해 줘",
    tools = [{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
        }], 
        tool_choice="required" 
)

print(response)

Response(id='resp_685cd15afe7c819b9912a4edb6e782490e6c65f1a9d8c021', created_at=1750913371.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_685cd15c4fa8819b835972be8a844e3a0e6c65f1a9d8c021', queries=['소켓 만드는 방법', 'how to create a socket', '소켓 생성 순서', '소켓 프로그래밍', '소켓 예시'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_685cd15ffd00819bb012cf8b492938860e6c65f1a9d8c021', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-2RmWSjR5STnJxWVZVnE6aW', filename='howto-sockets.pdf', index=1090, type='file_citation')], text="소켓 만드는 방법을 간단하게 설명해드리면 다음과 같습니다:\n\n### 1. 소켓 생성\n먼저, 소켓 객체를 만들어야 합니다. (파이썬 예시)\n```python\nimport socket\ns = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # INET(IPv4), STREAM(TCP)\n```\n- `AF_INET`: IPv4 주소 체계 사용\n- `SOCK_STREAM`: TCP 사용\n\n### 2. 서버 소켓: 주소와 포트 바인딩 (서버일 때만!)\n```python\ns.b

In [14]:
print(response.output_text)

소켓 만드는 방법을 간단하게 설명해드리면 다음과 같습니다:

### 1. 소켓 생성
먼저, 소켓 객체를 만들어야 합니다. (파이썬 예시)
```python
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # INET(IPv4), STREAM(TCP)
```
- `AF_INET`: IPv4 주소 체계 사용
- `SOCK_STREAM`: TCP 사용

### 2. 서버 소켓: 주소와 포트 바인딩 (서버일 때만!)
```python
s.bind(('호스트이름또는IP', 포트번호))
```

### 3. 서버 소켓: 연결 대기상태로 변경
```python
s.listen(5)  # 5는 대기 커넥션 수
```

### 4. 클라이언트 소켓: 서버에 연결
```python
s.connect(('서버주소', 포트번호))
```

### 5. 데이터 송수신
송신:
```python
s.send(보낼데이터)
```
수신:
```python
data = s.recv(버퍼크기)
```

### 6. 소켓 종료
```python
s.close()
```

#### 간단 예시(클라이언트):
```python
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('example.com', 80))
s.send(b'Hello')
data = s.recv(1024)
s.close()
```

#### 간단 예시(서버):
```python
import socket
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 12345))
server.listen(5)
client, addr = server.accept()
client.send(b'Hello')
client.close()
server.close()
```

이렇게 소켓은 *